In [1]:
import pandas as pd

In [13]:
# source: https://www.rva.gov/assessor-real-estate/data-request

files = {'2012ytdTransfers.xls':'2012 Transfers',
    '2013ytdTransfers.xls':'2013 Transfers',
    '2014ytdTransfers.xls':'2014 Transfers',
    '2015ytdTransfers.xlsx':'_2015Transfers',
    '2016ytdTransfers.xlsx':'_2016Transfers',
    '2017ytdTransfers.xlsx':'_2017Transfers',
    '2018ytdTransfers.xlsx':'_2018Transfers',
    '2019Transfers.xlsx':'_2019Transfers',
    'Assessor_Transfers_2020_Excel.xlsx':'_2020Transfers',
    'Assessor_Transfers_2021.xlsx':'Assessor_Transfers_2021',
    'Assessor_Transfers_2022_20220418.xlsx':'Assessor_Transfers_2022'}


In [22]:
# Load all files into one DF
for f in files.keys():
    dfs = pd.read_excel(f'data/{f}', sheet_name=None)
    tdf = dfs[files[f]]
    if 'df' in vars():
        df = df.append(tdf, ignore_index=True)
    else:
        df = tdf

In [24]:
df.shape

(55053, 20)

In [ ]:
# Find messy columns (with multiple types)
#df.applymap(type).nunique()
#df['Story'].map(type).unique()

In [94]:
# Clean up types - most of these have alpha chars in at least some rows, so we cast to strings
df['Deed_page'] = df['Deed_page'].astype('str').str.strip()
df['Card'] = df['Card'].astype('str').str.strip()
df['Class_desc'] = df['Class_desc'].astype('str').str.strip()
df['NBHDname'] = df['NBHDname'].astype('str').str.strip()
df['Transfer_comment'] = df['Transfer_comment'].astype('str').str.strip()
df['Story'] = pd.to_numeric(df['Story'], errors='coerce')
df['TransferComment'] = df['TransferComment'].astype('str').str.strip()
df['Grantee'] = df['Grantee'].str.strip()

In [95]:
#df.to_parquet('data/transfers_since_2013.parquet')
df = pd.read_parquet('data/transfers_since_2013.parquet')

In [96]:
df['PropClass'] = pd.to_numeric(df['Class_desc'].str[:3], errors='coerce')

In [97]:
# filter to SFH only
# 43081 out of 55053 transfers were single-family homes
df = df.loc[df['PropClass'].isin([101,110,115,120,130,150])]

In [99]:
# Top buyers
df.loc[df['Transfer_date'].dt.year >= 2020]\
.groupby('Grantee').agg({'Consideration':'mean', 'Parcel_ID':'count'}).loc[:,['Consideration','Parcel_ID']].sort_values('Parcel_ID',ascending=False)

,Consideration,Parcel_ID
Grantee,,
NVR INC,2.749615e+05,180
CAVA CAPITAL LLC,1.558041e+06,91
RANDOLPH HOMES LLC,9.281889e+05,45
RED 144 LLC,5.078439e+06,42
WATCHTOWER HOMES AND,1.008400e+05,35
...,...,...
GOLDSMITH JASON F,3.650000e+05,1
GOLDSMITH GLENN E II TRUST,1.750000e+05,1
GOLDSCHEITTER GALEN MICHAEL,3.500000e+05,1


In [110]:
# Top sellers
df.loc[df['Transfer_date'].dt.year >= 2020]\
    .groupby('Grantor').agg({'Consideration':'mean', 'Parcel_ID':'count'}).loc[:,['Consideration','Parcel_ID']]\
    .sort_values('Parcel_ID',ascending=False)\
    .head(10)

,Consideration,Parcel_ID
Grantor,,
NVR INC,3.702662e+05,151
MANCHESTER TOWN CENTER LLC,4.360089e+05,90
WESTLAKE LLC,1.152216e+05,88
RVA75 19 LLC,1.863270e+06,71
CAVA CAPITAL LLC,2.991836e+05,61
STANLEY MARTIN COMPANIES LLC,3.171017e+05,59
ECK ENTERPRISES INC,3.254114e+06,44
INTERMONT 1 LLC,4.264014e+06,35
SOUTHSIDE COMMUNITY DEVELOPMENT,1.785536e+05,28


* NVR INC = Ryan Homes, NVHomes and Heartland Homes


In [108]:
# Most parcels transferred in a single sale
df.groupby(['Deed_book','Deed_page','Grantee']).agg({'Consideration':'mean', 'Parcel_ID':'count'})\
    .sort_values(['Parcel_ID','Consideration'],ascending=[False,False]).head(10)

Consideration  Parcel_ID
Deed_book Deed_page Grantee                                         
ID2018    12023     GUSNPENEL INC                  900000         51
ID2021    15780     RED 144 LLC                   5078439         42
ID2016    2080      GULNORA LLC AND                 15000         29
ID2021    909       CAVA CAPITAL LLC              4686000         28
          15784     WHITE 144 LLC                 3407117         23
ID2015    7103      THE BEVERLEY GROUP LLC        1731837         21
ID2016    20632     NORDHEIM STEVEN L              674900         21
ID2021    22883     RANDOLPH HOMES LLC            1875000         20
          15773     BLUE 144 LLC                  2130523         17
ID2013    1836      ANDRE PROPERTIES LLC                0         16

In [111]:
df.loc[df['Prop_stree']=='6521 Vischer Road']

,Parcel_ID,Card,Class_desc,NBHDno,NBHDname,district_number,Prop_stree,Transfer_date,Consideration,Transfer_comment,...,Deed_page,Grantor,Grantee,Story,YearBlt,FinArea,TransferComment,YrBuilt,FinSize,PropClass
52419,C0050351025,nan,101 Single Family Vacant (R1-R7),160.0,Granite Hill,412.0,6521 Vischer Road,2021-09-28,86000,nan,...,28009,WESTLAKE LLC,NVR INC,NaN,NaN,NaN,nan,NaN,NaN,101.0
54070,C0050351025,R01,101 Single Family Vacant (R1-R7),160.0,Granite Hill,412.0,6521 Vischer Road,2022-01-19,426935,nan,...,1328,NVR INC,BAGDASIAN MARY ROSA,2.0,NaN,NaN,nan,2021.0,2340.0,101.0


We have a couple of markers for identifying flips vs. new housing:
If YrBuilt is <2 years before Transfer_date, it's new housing
If YrBuilt is NaN, it's probably a vacant lot